# FACE Hyperparameter Results

In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '../..'))

%load_ext autoreload
%autoreload 2

import joblib
import pandas as pd
import numpy as np
from sklearn import neighbors
from sklearn import model_selection

import matplotlib.pyplot as plt
import seaborn as sns
from models import model_interface, model_loader, model_constants
from data import data_loader
from data.adapters import continuous_adapter
from scripts import fit_kde

In [2]:
RECOURSE_METHOD = 'face'
RESULTS_DIR = '../../experiment_results/face/face_hyperparam'

# Preliminaries -- load everything

In [3]:
DATASET, DATASET_INFO = data_loader.load_data(data_loader.DatasetName('credit_card_default'), split="train")
MODEL = model_loader.load_model(model_constants.ModelType('logistic_regression'), data_loader.DatasetName('credit_card_default'))
ADAPTER = continuous_adapter.StandardizingAdapter(
    label_column = DATASET_INFO.label_column, positive_label=DATASET_INFO.positive_label
).fit(DATASET)

DROP_COLUMNS = ['step_id', 'path_id', 'run_id', 'batch_id']  # columns which are convenient to drop from the path_df

config_df = pd.read_csv(os.path.join(RESULTS_DIR, 'experiment_config_df.csv'))
path_df = pd.read_csv(os.path.join(RESULTS_DIR, f'{RECOURSE_METHOD}_paths_df.csv'))
config_df

,batch_id,run_id,run_seed,confidence_cutoff,counterfactual_mode,dataset_name,distance_threshold,graph_directory,max_iterations,model_type,noise_ratio,num_paths,rescale_ratio,split,elapsed_recourse_seconds
0,29,894,3069,0.7,True,credit_card_default,1.00,recourse_methods/face_graphs,50,logistic_regression,NaN,5,NaN,val,0.828081
1,59,1780,3363,0.8,True,credit_card_default,2.50,recourse_methods/face_graphs,50,logistic_regression,NaN,5,NaN,val,2.156598
2,18,568,6779,0.6,True,credit_card_default,2.50,recourse_methods/face_graphs,50,logistic_regression,NaN,4,NaN,val,3.332527
3,66,1982,1701,0.9,True,credit_card_default,1.00,recourse_methods/face_graphs,50,logistic_regression,NaN,2,NaN,val,0.395447
4,11,345,3439,0.6,True,credit_card_default,1.50,recourse_methods/face_graphs,50,logistic_regression,NaN,2,NaN,val,1.262999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,6,203,211,0.6,True,credit_card_default,1.00,recourse_methods/face_graphs,50,logistic_regression,NaN,2,NaN,val,0.309175
2396,48,1458,2020,0.8,True,credit_card_default,1.00,recourse_methods/face_graphs,50,logistic_regression,NaN,4,NaN,val,0.300586
2397,31,954,3069,0.7,True,credit_card_default,1.50,recourse_methods/face_graphs,50,logistic_regression,NaN,2,NaN,val,0.663127
2398,24,735,3439,0.7,True,credit_card_default,0.75,recourse_methods/face_graphs,50,logistic_regression,NaN,5,NaN,val,0.215227


## Load or Fit a KDE

In [4]:
KDE_DIRECTORY = '../../saved_models/kde/credit_card_default_kde.joblib'

if os.path.exists(KDE_DIRECTORY):
    KDE = joblib.load(KDE_DIRECTORY)
else:
    KDE = fit_kde.fit_kde('credit_card_default', KDE_DIRECTORY)

# Analyze the results

In [8]:
SPARSITY_EPSILON = 1e-5

def get_poi_cfes(path_df: pd.DataFrame):
    """Isolate the POIs (Points of Interest) and CFEs (Counterfactual Examples) from the full path results.
    
    POIs and CFEs are listed in the order they originally appear in. There is one POI and one CFE
    for every path that appears in the DataFrame."""
    pathscopy = path_df.copy()
    pathscopy['next_step_id'] = 0
    pathscopy.loc[:,'next_step_id'].iloc[0:-1] = pathscopy.loc[:,'step_id'].iloc[1:]
    cfes = pathscopy[pathscopy.step_id >= pathscopy.next_step_id].drop(columns='next_step_id')
    return pathscopy[pathscopy.step_id == 0].drop(columns='next_step_id'), cfes

def get_sparsity(path: pd.DataFrame):
    """Returns the maximum number of features changed in any single iteration
    along the path."""
    if path.shape[0] == 1:
        return np.nan
    path_sparsity = np.zeros(path.shape[0])
    for i in range(1, path.shape[0]):
        path_sparsity[i] = (np.abs(path.iloc[i] - path.iloc[i - 1]) > SPARSITY_EPSILON).sum()
    return np.max(path_sparsity)

def get_path_length(path: pd.DataFrame):
    """Returns the sum of euclidean distances along the path."""
    total = 0
    for i in range(1, path.shape[0]):
        total += np.linalg.norm(path.iloc[i] - path.iloc[i - 1])
    if total == 0:
        return np.nan
    return total

def get_cfe_distance(path: pd.DataFrame):
    """Returns the euclidean distance between the first and last points in the path."""
    if len(path) == 1:
        return np.nan
    return np.linalg.norm(path.iloc[-1] - path.iloc[0])


def analyze_paths(paths: pd.DataFrame, poi_kdes, cfe_kdes, cfe_probs, config_df):
    """Returns a DataFrame containing per-path results.
    
    Each row corresponds to a specific path. Each column is a result metric.
    
    Args:
        paths: The path_df DataFrame to analyze.
        poi_kdes: The KDE scores for the POIs.
        cfe_kdes: The KDE scores for the CFEs.
        config_df: The experiment_config_df for the experiment."""
    columns = ['run_id', 'path_id', 'success', 'proximity', 'path_length',
               'iteration_count', 'poi_density', 'cfe_density', 
               'actual_sparsity']
    col_idx = {}
    for i, col in enumerate(columns):
        col_idx[col] = i

    results = np.zeros((len(poi_kdes), len(columns)))

    i = 0
    for run_id in paths.run_id.unique():
        run_paths = paths[paths.run_id == run_id]
        for path_id in run_paths.path_id.unique():
            path = ADAPTER.transform(run_paths[run_paths.path_id == path_id].drop(columns=DROP_COLUMNS))
            results[i,col_idx['run_id']] = run_id
            results[i,col_idx['path_id']] = path_id

            desired_proba = config_df[config_df.run_id == run_id].confidence_cutoff.iloc[0]
            actual_proba = cfe_probs[i]

            results[i,col_idx['success']] = 1 if actual_proba >= desired_proba else 0
            results[i,col_idx['path_length']] = get_path_length(path)
            results[i,col_idx['iteration_count']] = len(path)
            results[i,col_idx['proximity']] = get_cfe_distance(path)
            results[i,col_idx['poi_density']] = poi_kdes[i]
            results[i,col_idx['cfe_density']] = cfe_kdes[i]
            results[i,col_idx['actual_sparsity']] = get_sparsity(path)
            i += 1

    return pd.DataFrame(data=results, columns=columns)

pois, cfes = get_poi_cfes(path_df)
poi_kdes = KDE.score_samples(ADAPTER.transform(pois.drop(columns=DROP_COLUMNS)))
cfe_kdes = KDE.score_samples(ADAPTER.transform(cfes.drop(columns=DROP_COLUMNS)))
cfe_probs = MODEL.predict_pos_proba(cfes.drop(columns=DROP_COLUMNS)).to_numpy()

results = analyze_paths(path_df, poi_kdes, cfe_kdes, cfe_probs, config_df)
results = results.merge(config_df, how='left', on='run_id').drop(
    columns=['dataset_name', 'max_iterations', 'model_type', 'noise_ratio',
             'rescale_ratio', 'run_seed', 'split', 'graph_directory', 'counterfactual_mode'])  # uninteresting columns
results

/tmp/ipykernel_1651/1445753859.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pathscopy.loc[:,'next_step_id'].iloc[0:-1] = pathscopy.loc[:,'step_id'].iloc[1:]


,run_id,path_id,success,proximity,path_length,iteration_count,poi_density,cfe_density,actual_sparsity,batch_id,confidence_cutoff,distance_threshold,num_paths,elapsed_recourse_seconds
0,894.0,0.0,0.0,NaN,NaN,1.0,-88.107767,-88.107767,NaN,29,0.7,1.00,5,0.828081
1,894.0,1.0,0.0,NaN,NaN,1.0,-88.107767,-88.107767,NaN,29,0.7,1.00,5,0.828081
2,894.0,2.0,0.0,NaN,NaN,1.0,-88.107767,-88.107767,NaN,29,0.7,1.00,5,0.828081
3,894.0,3.0,0.0,NaN,NaN,1.0,-88.107767,-88.107767,NaN,29,0.7,1.00,5,0.828081
4,894.0,4.0,0.0,NaN,NaN,1.0,-88.107767,-88.107767,NaN,29,0.7,1.00,5,0.828081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,735.0,2.0,0.0,NaN,NaN,1.0,1.339605,1.339605,NaN,24,0.7,0.75,5,0.215227
7196,735.0,3.0,0.0,NaN,NaN,1.0,1.339605,1.339605,NaN,24,0.7,0.75,5,0.215227
7197,735.0,4.0,0.0,NaN,NaN,1.0,1.339605,1.339605,NaN,24,0.7,0.75,5,0.215227
7198,1995.0,0.0,0.0,NaN,NaN,1.0,1.339605,1.339605,NaN,66,0.9,1.00,2,0.299082


# Choosing metrics

We must select values for:
* num_paths
* confidence_cutoff
* distance_threshold

Can we use the metrics chosen by StEP?
* num_paths=3
* confidence_cutoff=0.7

Let's see what the best-performing parameter settings look like.

In [46]:
DROP_METRICS = ['run_id', 'elapsed_recourse_seconds', 'negative_cfe_density',
                'path_id', 'batch_id', 'actual_sparsity', 'negative_success']

results['negative_cfe_density'] = -results['cfe_density']
results['negative_success'] = -results['success']
results.groupby('batch_id', as_index=False).mean().sort_values(
    ['negative_success', 'path_length', 'proximity', 'negative_cfe_density', 'iteration_count']).iloc[:5].drop(
        columns=DROP_METRICS)

,success,proximity,path_length,iteration_count,poi_density,cfe_density,confidence_cutoff,distance_threshold,num_paths
15,0.700000,3.541706,3.541706,1.700000,-7.499876,-4.108164,0.6,2.5,1.0
16,0.700000,3.650092,3.650092,1.700000,-7.499876,-4.076103,0.6,2.5,2.0
17,0.688889,3.646413,3.646413,1.688889,-7.499876,-4.094771,0.6,2.5,3.0
18,0.683333,3.656454,3.656454,1.683333,-7.499876,-4.060181,0.6,2.5,4.0
19,0.680000,3.691335,3.691335,1.680000,-7.499876,-4.069774,0.6,2.5,5.0


Whoops! FACE fails pretty frequently. Let's look at that.

If FACE failed, then its iteration count == 1 (because there is only the POI in the path).

We can use this to calculate the success rate for a given parameter setting.

In [47]:
def myfunc(df: pd.DataFrame):
    success_ratio = len(df[df.iteration_count > 1]) / len(df)
    return pd.Series([success_ratio], index=['success_ratio'])

results.groupby('distance_threshold').apply(myfunc)

,success_ratio
distance_threshold,
0.75,0.000000
1.00,0.016667
1.50,0.405000
2.50,0.568333


In [48]:
results.groupby('confidence_cutoff').apply(myfunc)

,success_ratio
confidence_cutoff,
0.6,0.321111
0.7,0.263333
0.8,0.241667
0.9,0.163889


In [49]:
results.groupby('num_paths').apply(myfunc)

,success_ratio
num_paths,
1,0.279167
2,0.257292
3,0.248611
4,0.243229
5,0.240000


We can see that FACE's success is strongly effected by the graph distance_threshold. We will set distance_threshold = 2.5 for this reason.

## Back to the best-performing parameters...

In [52]:
good_results = results[results.distance_threshold == 2.5]



good_results.groupby('batch_id', as_index=False).mean().sort_values(
    ['negative_success', 'path_length', 'proximity', 'negative_cfe_density', 'iteration_count']).iloc[:10].drop(
        columns=DROP_METRICS)

,success,proximity,path_length,iteration_count,poi_density,cfe_density,confidence_cutoff,distance_threshold,num_paths
0,0.700000,3.541706,3.541706,1.700000,-7.499876,-4.108164,0.6,2.5,1.0
1,0.700000,3.650092,3.650092,1.700000,-7.499876,-4.076103,0.6,2.5,2.0
2,0.688889,3.646413,3.646413,1.688889,-7.499876,-4.094771,0.6,2.5,3.0
3,0.683333,3.656454,3.656454,1.683333,-7.499876,-4.060181,0.6,2.5,4.0
4,0.680000,3.691335,3.691335,1.680000,-7.499876,-4.069774,0.6,2.5,5.0
5,0.600000,4.060799,4.060799,1.600000,-7.499876,-5.177196,0.7,2.5,1.0
6,0.600000,4.096632,4.096632,1.600000,-7.499876,-4.860327,0.7,2.5,2.0
7,0.600000,4.127089,4.127089,1.600000,-7.499876,-5.001862,0.7,2.5,3.0
8,0.583333,4.058119,4.058119,1.583333,-7.499876,-5.192546,0.7,2.5,4.0
9,0.573333,4.007029,4.007029,1.573333,-7.499876,-5.244297,0.7,2.5,5.0


# Final Parameters

We see that success is effected, in order of significance, by:
* distance_threshold
* confidence_cutoff
* num_paths

The StEP parameters perform tied for 5th-best and succeed 10% less frequently
than the optimal parameters. To keep consistency with DiCE and StEP, we choose
these parameters.

* confidence_cutoff: 0.7
* num_paths: 3
* distance_threshold: 2.5